In [175]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [176]:
df = pd.read_csv('SpotifyFeatures.csv')  # load dataset
df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


In [177]:
# pick audio features

# acousticness, danceability, energy, instrumentalness, liveness, loudness, speechinevalence,

In [178]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
genre_encoded = encoder.fit_transform(df[['genre']])

feat_cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'tempo', 'valence']

In [179]:
from sklearn.preprocessing import normalize
#0.7, 1.3, 1.5, 0.8, 1.8, 1.2
feature_weights = np.array([0.7, 1.3, 1.5, 0.8, 1.8, 1.2])

X = StandardScaler().fit_transform(df[feat_cols])

# Apply weights
X_weighted = X * feature_weights

genre_weight = 0.3
genre_weighted = genre_encoded * genre_weight

X_final = np.hstack((X_weighted, genre_weighted))

In [180]:
X_final

array([[ 0.47836341, -1.15821522,  1.93036191, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24182703,  0.24959179,  0.94537395, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15119886,  0.76088489, -2.50493071, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.05056951, -0.2617013 , -0.86518196, ...,  0.3       ,
         0.        ,  0.        ],
       [-0.21025704,  1.33521412,  0.75748607, ...,  0.3       ,
         0.        ,  0.        ],
       [-0.53523064,  1.42626631, -0.57480979, ...,  0.3       ,
         0.        ,  0.        ]])

In [181]:
# Top K songs for query song
def find_top_similar_songs(query_song_index, features_matrix, song_names, artist_names, genre_names, k=5):
  query_features = features_matrix[query_song_index].reshape(1, -1)
  similarities = cosine_similarity(query_features, features_matrix).flatten()

  # Exclude the query song itself
  similarities[query_song_index] = -1

  # Sort all songs by similarity (descending)
  sorted_indices = np.argsort(similarities)[::-1]

  query_name = song_names.iloc[query_song_index]
  seen_names = set([query_name])  # track seen song names
  results = []

  for idx in sorted_indices:
      name = song_names.iloc[idx]
      artist = artist_names.iloc[idx]
      genre = genre_names.iloc[idx]
      if name not in seen_names:  # ensures no duplicates
          results.append((name, artist, genre, similarities[idx]))
          seen_names.add(name)
      if len(results) == k:
          break

  return results

song_name_to_find = "SLOW DANCING IN THE DARK - Loud Luxury Remix"
index = df[df['track_name'] == song_name_to_find].index[0]

print("Top 5 songs similar to this song: " + df["track_name"][index])
find_top_similar_songs(index, X_final , df["track_name"], df["artist_name"], df["genre"], 10)

Top 5 songs similar to this song: SLOW DANCING IN THE DARK - Loud Luxury Remix


[('Colors - Coca-Cola® Anthem, 2018 FIFA World CupTM',
  'Jason Derulo',
  'R&B',
  np.float64(0.9877111280361719)),
 ('OLLA (Only Lovers Left Alive)',
  'Jhene Aiko',
  'R&B',
  np.float64(0.9873547489315203)),
 ('Colors', 'Jason Derulo', 'R&B', np.float64(0.9855176394872407)),
 ('Blue Ribbon', 'Jessie Reyez', 'R&B', np.float64(0.985386292004289)),
 ('Watch Me', 'Nick Jonas', 'R&B', np.float64(0.9841268740293391)),
 ('Lose My Breath', "Destiny's Child", 'R&B', np.float64(0.9823661890406339)),
 ('Close To Me (with Diplo) (feat. Swae Lee) - Felix Cartal Remix',
  'Ellie Goulding',
  'R&B',
  np.float64(0.9819297886891225)),
 ('Not Myself Tonight',
  'Christina Aguilera',
  'R&B',
  np.float64(0.9810345021203316)),
 ('NBD (Phlegmatic Dogs Remix)',
  'Boombox Cartel',
  'Electronic',
  np.float64(0.9802445823233001)),
 ("He Can't Love U", 'Jagged Edge', 'R&B', np.float64(0.980149338720999))]

In [182]:
df['genre'].unique()

array(['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Children’s Music', 'Rap', 'Indie',
       'Classical', 'Pop', 'Reggae', 'Reggaeton', 'Jazz', 'Rock', 'Ska',
       'Comedy', 'Soul', 'Soundtrack', 'World'], dtype=object)

Another approach below

In [183]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

In [184]:
df = pd.read_csv("SpotifyFeatures.csv")   # update name if needed
df = df.dropna(subset=['track_name', 'artist_name', 'genre'])

In [185]:
feat_cols = [
    'acousticness', 'danceability', 'energy',
    'instrumentalness', 'tempo', 'valence'
]

In [186]:
scaler = StandardScaler()
X_audio = scaler.fit_transform(df[feat_cols])

In [187]:
feature_weights = np.array([0.7, 1.3, 1.5, 0.8, 1.8, 1.2])
X_audio_weighted = X_audio * feature_weights

In [188]:
tfidf = TfidfVectorizer()
genre_tfidf = tfidf.fit_transform(df["genre"])

# Reduce genre dimensions
svd = TruncatedSVD(n_components=5)
genre_emb = svd.fit_transform(genre_tfidf)

# Apply genre weight
genre_emb = genre_emb * 0.3

In [189]:
X_combined = np.hstack((X_audio_weighted, genre_emb))

In [190]:
#pca = PCA(n_components=10)
#X_pca = pca.fit_transform(X_combined)

In [191]:
X_final = X_combined

In [192]:

def recommend(song_name, df, features_matrix, k=10):
  matches = df[df['track_name'].str.lower() == song_name.lower()]
  if matches.empty:
    raise ValueError(f"Song '{song_name}' not found in dataset.")

  query_index = matches.index[0]
  query_vector = features_matrix[query_index].reshape(1, -1)

  similarities = cosine_similarity(query_vector, features_matrix).flatten()

  # Set its own similarity to -1 so it's excluded
  similarities[query_index] = -1

  # Rank by similaritiy
  top_indices = np.argsort(similarities)[::-1][:k]

  results = []
  seen = set()

  for idx in top_indices:
    name = df["track_name"].iloc[idx]
    if name not in seen:
      results.append({
          "track_name": name,
          "artist_name": df["artist_name"].iloc[idx],
          "genre": df["genre"].iloc[idx],
          "similarity": float(similarities[idx])
      })
      seen.add(name)

  return results

In [193]:
song = "SLOW DANCING IN THE DARK - Loud Luxury Remix"

recommendations = recommend(song, df, X_final, k=10)

for r in recommendations:
  print(r)

{'track_name': 'SLOW DANCING IN THE DARK - Loud Luxury Remix', 'artist_name': 'Joji', 'genre': 'Pop', 'similarity': 0.9992551240895547}
{'track_name': 'NBD (Phlegmatic Dogs Remix)', 'artist_name': 'Boombox Cartel', 'genre': 'Electronic', 'similarity': 0.9913991286303224}
{'track_name': 'Go', 'artist_name': 'The Chemical Brothers', 'genre': 'Dance', 'similarity': 0.9913111487390492}
{'track_name': 'Werk', 'artist_name': 'CID', 'genre': 'Dance', 'similarity': 0.9906625976485642}
{'track_name': 'Simple (feat. Victoria Zaro)', 'artist_name': 'Autograf', 'genre': 'Soul', 'similarity': 0.9896551801178587}
{'track_name': 'Days Are Gone', 'artist_name': 'HAIM', 'genre': 'Folk', 'similarity': 0.9896125353073391}
{'track_name': 'Drummer Boy', 'artist_name': 'Justin Bieber', 'genre': 'Dance', 'similarity': 0.9893393481667996}
